Attempting to scrape information from a webpage whose output changes dynamically as we enter text in a field

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

Atttempting to get rid of initial popup:

In [11]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')

driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


Experimenting to find the right web element and regex used to extract the data:

In [13]:
ID = "100030"

provider_list = [ID]

In [14]:
driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)

In [ ]:
table1 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody")[0].text
table2 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody")[0].text
table = table1 + "\n" + table2
print(table)


In [10]:
# print(table.split("\n"))
print(len(table.split("\n")))
column_names = ["ProviderID"]
for i in table.split("\n"):
    column_names.append(i[:i.rfind(' ')])
# print(column_names)
for i in range(len(column_names)):
    column_names[i] = column_names[i].replace(' ','_')
print(column_names)

60
['ProviderID', 'Fiscal_year_begin_date', 'Report_date', 'Termination_Date', 'Waiver_code', 'Intermediary_number', 'Provider_type', 'Current_Census_division', 'Change_code_for_reclass', 'Actual_Geographic_MSA', 'Wage_Index_MSA', 'Standardized_amount_MSA', 'Sole_community_or_Medicare', 'LUGAR', 'Temporary_relief_indicator', 'Federal_PPS_Blend_indicator', 'Facility_specific_rate', 'Cost_of_living_adjustment', 'Intern_to_bed_ratio', 'Bed_size', 'Cost-to-charge_ratio', 'Case_mix_index', 'SSI_ratio', 'Medicaid_ratio', 'Provider_PPS_period', 'Special_provider_update_factor', 'Operating_DSH_percent', 'Fiscal_year_end_date', 'Special_wage_index', 'Special_payment_indicator', 'Hospital_quality_indicator', 'Geographic_location_CBSA', 'Reclassification_CBSA', 'Standardized_amount_CBSA', 'Total_pass_through_amount_and_miscellaneous', 'Pass_through_amount_capital', 'Pass_through_amount_direct', 'Pass_through_amount_organ', 'Capital_PPS_pay_code', 'Capital_hospital_specific_rate', 'Capital_old_har

In [37]:
provider_list = [ID]
for i in table.split("\n"):
    provider_list.append(i[i.rfind(' ')+1:])
print(len(provider_list))

61


Importing the inpatient provider numbers and checking to see if it loaded correctly:

In [5]:
inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")
inpatient.head(5)

,Inpatient,CCN
0,NaN,10001
1,NaN,10005
2,NaN,10006
3,NaN,10007
4,NaN,10008


Alternatively, if using all the Provider IDs:

*Obtained by querying* `SELECT [Federal_Provider_Number] FROM [ProviderDataRegistry].[dbo].[CMS_ProviderInfo]`

In [22]:
all_providers = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")
all_providers.head(5)

,Federal_Provider_Number
0,015009
1,015010
2,015012
3,015014
4,015015


Run this block to perform web scraping:

In [34]:
# one hour for ~3000 provider ID_s

# Changing the data to all the providers
# inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")
all_providers = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")

try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

ID_list = all_providers["Federal_Provider_Number"].to_list()
ID_list = [str(x) for x in ID_list]
for i in range(len(all_providers)):
    while(len(ID_list[i]) < 6):  
        ID_list[i] = "0" + ID_list[i]
     
final_list = []
df = pd.DataFrame([])
count = 1
l = len(ID_list)

recurring_error = 0
error_IDs = []

for ID in ID_list:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)

    recurring_error = 0
    while True:
        try:
            if recurring_error == 5:
                 break;
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            break;
        except Exception:
            recurring_error += 1
            time.sleep(4)

    if recurring_error == 5:
        error_IDs.append(ID)
        continue

    recurring_error = 0
    while True:
        try:
            if recurring_error == 5:
                 break;
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            break;
        except Exception:
            recurring_error += 1
            time.sleep(4)
    
    if recurring_error == 5:
        error_IDs.append(ID)
        continue
    
    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print("\ (•◡•) /   Extracted data from Provider ID: " + ID + "!   \ (•◡•) /", end = "  ")
    print("  <<"+str(count)+" out of "+str(l)+" provider's info taken!>>", end = "\r")
    count += 1
    

df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


KeyboardInterrupt: 

In [26]:
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,015009,07/01/2021,03/31/2022,--,N,05901,38,5,N,01,...,0,0,--,$0.00,--,0,--,01,0,--
1,015010,07/01/2021,03/31/2022,--,N,10111,38,5,--,01,...,0,0,--,$0.00,--,0,--,01,0,--
2,015012,10/01/2021,03/31/2022,--,N,10111,38,5,--,01,...,0,0,--,$0.00,--,0,--,01,0,--
3,015014,07/01/2021,03/31/2022,--,N,05901,38,5,N,1000,...,0,0,--,$0.00,--,0,--,01,0,--
4,015015,07/01/2021,03/31/2022,--,N,05901,38,5,N,1000,...,0,0,--,$0.00,--,0,--,01,0,--
5,015016,07/01/2021,03/31/2022,--,N,10111,38,5,--,3440,...,0,0,N,$0.00,--,0,--,01,0,--
6,015019,01/01/2021,03/31/2022,--,N,04011,38,6,--,5240,...,0,0,--,$0.00,--,0,01051,01,0,--


In [17]:
for i in range(len(final_list)):
    if len(final_list[i]) != 61:
        final_list[i] = final_list[i][:61] 
        print(final_list[i])



['050137', '01/01/2022', '03/31/2022', '--', 'N', '01911', '07', '9', '--', '0000', '0000', '0000', '--', '--', '--', '--', '$2,788.67', '1', '0.0114', '183', '0.338', '1.3803', '0.0871', '0.1659', '--', '--', '--', '12/31/2016', '--', 'Y', '1', '31084', '37100', '05', '$3.78', '--', '$3.78', '--', 'C', '384.57', '0', '0', '0.008', '--', '0.0233', '0', '--', '--', 'N', '0', '1', '1', '0', 'N', '$4,029.72', '--', '0', '06037', '05', '0', '--']


In [16]:
x = [1,2,3,4,5]
print(x[:3])

[1, 2, 3]


In [35]:
print(error_IDs)

['01A193', '01A208', '025010', '025015', '025018', '025019', '025020', '025021', '025024', '025025', '025026', '025027', '025028', '025030', '025031', '025032', '025034', '025035', '025036', '025037', '025038', '025039', '035003', '035004', '035014', '035016', '035059', '035062', '035064', '035068', '035070', '035071', '035072', '035073', '035074', '035076', '035083', '035084', '035085', '035086', '035087', '035088', '035091', '035092', '035093', '035094', '035095', '035096', '035097', '035099', '035100', '035101', '035103', '035105', '035106', '035107', '035110', '035111', '035112', '035114', '035116', '035117', '035118', '035120', '035121', '035125', '035126', '035127', '035130', '035131', '035132']


In [21]:
df.to_csv("test.csv")

In [20]:
print(final_list[0])

['010001', '10/01/2021', '03/31/2022', '--', 'N', '10111', '07', '5', 'Y', '2180', '01', '01', '--', '--', '--', '--', '$1,918.13', '1', '0.0494', '322', '0.181', '--', '0.1029', '0.1989', '--', '--', '--', '09/30/2022', '0.856', '2', '1', '20020', '18880', '01', '$106.82', '--', '$46.13', '--', 'C', '0', '0', '0', '0.012', '--', '0.1136', '0', '--', '--', 'N', '0', '1', '0.9903', '0', 'N', '$672.33', '--', '0', '01069', '01', '0', '--']
